# Day 5
## Part 1

In [1]:
#r "nuget: FSharp.UMX"

open FSharp.UMX

type [<Measure>] seed
type [<Measure>] soil
type [<Measure>] fertilizer
type [<Measure>] water
type [<Measure>] light
type [<Measure>] temperature
type [<Measure>] humidity
type [<Measure>] location

type MapRange<[<Measure>]'source, [<Measure>]'destination> = 
    { DestinationStart : int64<'destination>; SourceStart : int64<'source>; RangeSize : int64 }

type InputSection =
    | SeedList of list<int64<seed>>
    | SeedToSoilMap of MapRange<seed, soil>[]
    | SoilToFertilizerMap of MapRange<soil, fertilizer>[]
    | FertilizerToWaterMap of MapRange<fertilizer, water>[]
    | WaterToLightMap of MapRange<water, light>[]
    | LightToTemperatureMap of MapRange<light, temperature>[]
    | TemperatureToHumidityMap of MapRange<temperature, humidity>[]
    | HumidityToLocationMap of MapRange<humidity, location>[]



Installed Packages FSharp.UMX, 1.1.0

In [2]:
#r "nuget:FParsec"

open FParsec

let space = pchar ' '
let newline = pchar '\n'
let int_list = sepBy pint64 space
let seedList = 
    (pstring "seeds: " >>. int_list .>> newline)
    |>> List.map UMX.tag<seed>
    |>> SeedList

let inline map_line<[<Measure>]'source, [<Measure>]'destination> = 
    pint64 .>> space .>>. pint64 .>> space .>>. pint64
    |>> fun ((a,b),c) -> { DestinationStart = UMX.tag<'destination> a; SourceStart = UMX.tag<'source> b; RangeSize = c }

let mapOrder (r : MapRange<'source, 'destination>) = r.SourceStart

let inline sortMap (ranges : MapRange<'source, 'destination> list) = 
    ranges
    |> List.sortBy mapOrder
    |> Array.ofList

let seedToSoil = 
    (pstring "seed-to-soil map:" >>. newline >>. sepEndBy1 map_line<seed,soil> newline)
    |>> (sortMap >> SeedToSoilMap)

let soilToFertilizer = 
    (pstring "soil-to-fertilizer map:" >>. newline >>. sepEndBy1 map_line<soil, fertilizer> newline)
    |>> (sortMap >> SoilToFertilizerMap)

let fertilizerToWater = 
    (pstring "fertilizer-to-water map:" >>. newline >>. sepEndBy1 map_line<fertilizer, water> newline)
    |>> (sortMap >> FertilizerToWaterMap)

let waterToLight =
    (pstring "water-to-light map:" >>. newline >>. sepEndBy1 map_line<water, light> newline)
    |>> (sortMap >> WaterToLightMap)

let lightToTemperature =
    (pstring "light-to-temperature map:" >>. newline >>. sepEndBy1 map_line<light, temperature> newline)
    |>> (sortMap >> LightToTemperatureMap)

let temperatureToHumidity =
    (pstring "temperature-to-humidity map:" >>. newline >>. sepEndBy1 map_line<temperature, humidity> newline)
    |>> (sortMap >> TemperatureToHumidityMap)

let humidityToLocation =
    (pstring "humidity-to-location map:" >>. newline >>. sepEndBy1 map_line<humidity, location> newline)
    |>> (sortMap >> HumidityToLocationMap)

let section =
    seedList <|>
    seedToSoil <|>
    soilToFertilizer <|>
    fertilizerToWater <|>
    waterToLight <|>
    lightToTemperature <|>
    temperatureToHumidity <|>
    humidityToLocation

let sections : Parser<InputSection list, unit> = sepBy section newline


Installed Packages FParsec, 1.1.1

In [3]:
open System
open System.Collections

type BinarySearchIndex = | Match of int | BeforeFirst | AfterLast of int | Between of int * int

let binarySearchBy<'T,'U when 'U :> IComparable<'U>> (sel : 'T -> 'U) (x:'U) (values:'T[]) =
    let compare = System.Collections.Generic.GenericComparer<'U>.Default.Compare
    let comparer = { new IComparer with member __.Compare(a, b) = compare (a :?> 'T |> sel, x) }
    let i = Array.BinarySearch(values, x, comparer)
    if i >= 0 then
        Match i
    else
        let iNext = ~~~i
        if iNext = 0 then
            BeforeFirst
        elif iNext = values.Length then
            AfterLast (iNext-1)
        else
            Between (iNext-1, iNext)

let binarySearch (x:'U) (values:'T[]) =
    let i = Array.BinarySearch(values, x)
    if i >= 0 then
        Match i
    else
        let iNext = ~~~i
        if iNext = 0 then
            BeforeFirst
        elif iNext = values.Length then
            AfterLast (iNext-1)
        else
            Between (iNext-1, iNext)

let applyMap (ranges: MapRange<'src, 'dst>[]) (src: int64<'src>) : int64<'dst> =
    let index = binarySearchBy mapOrder src ranges
    match index with
    | BeforeFirst -> UMX.cast<'src,'dst>src
    | AfterLast i
    | Match i
    | Between (i, _) -> 
        let range = ranges[i]
        let offset = UMX.untag<'src>(src - range.SourceStart)
        if offset < range.RangeSize then
            range.DestinationStart + UMX.tag<'dst>offset
        else
            UMX.cast<'src,'dst>src


In [4]:
#r "nuget:Hedgehog"
#r "nuget: FsUnit.Xunit"

open FsUnitTyped
open Hedgehog

property {
    let! array = 
        Gen.int64 (Range.linear 0 100)
        |> Gen.array (Range.linear 1 100) 
        |> Gen.map (fun arr -> Array.sort arr)

    let! target = Gen.int64 (Range.linear 0 100)

    let expected = binarySearch target array
    let actual = binarySearchBy id target array

    actual |> shouldEqual expected
    
    match actual with
    | Match i -> array.[i] |> shouldEqual target
    | BeforeFirst -> array[0] |> shouldBeGreaterThan target
    | AfterLast i -> array.[i] |> shouldBeSmallerThan target
    | Between (i1, i2) -> 
        array[i1] |> shouldBeSmallerThan target
        array[i2] |> shouldBeGreaterThan target

} |> Property.check

Installed Packages FsUnit.Xunit, 5.6.1 Hedgehog, 0.13.0

In [5]:
type Model = 
    {
        Seeds : list<int64<seed>>
        SeedToSoilMap : MapRange<seed, soil>[]
        SoilToFertilizerMap : MapRange<soil, fertilizer>[]
        FertilizerToWaterMap : MapRange<fertilizer, water>[]
        WaterToLightMap : MapRange<water, light>[]
        LightToTemperatureMap : MapRange<light, temperature>[]
        TemperatureToHumidityMap : MapRange<temperature, humidity>[]
        HumidityToLocationMap : MapRange<humidity, location>[]
    }

module Model =
    let empty = 
        {
            Seeds = []
            SeedToSoilMap = [||]
            SoilToFertilizerMap = [||]
            FertilizerToWaterMap = [||]
            WaterToLightMap = [||]
            LightToTemperatureMap = [||]
            TemperatureToHumidityMap = [||]
            HumidityToLocationMap = [||]
        }
    
    let applySection section model =
        match section with
        | SeedList seeds -> { model with Seeds = seeds }
        | SeedToSoilMap map -> { model with SeedToSoilMap = map }
        | SoilToFertilizerMap map -> { model with SoilToFertilizerMap = map }
        | FertilizerToWaterMap map -> { model with FertilizerToWaterMap = map }
        | WaterToLightMap map -> { model with WaterToLightMap = map }
        | LightToTemperatureMap map -> { model with LightToTemperatureMap = map }
        | TemperatureToHumidityMap map -> { model with TemperatureToHumidityMap = map }
        | HumidityToLocationMap map -> { model with HumidityToLocationMap = map }

    let fromSections sections =
        sections |> List.fold (fun model section -> applySection section model) empty

    let parse =
        sections
        |>> fromSections

    let getLocation model (seed: int64<seed>) =
        seed
        |> applyMap model.SeedToSoilMap
        |> applyMap model.SoilToFertilizerMap
        |> applyMap model.FertilizerToWaterMap
        |> applyMap model.WaterToLightMap
        |> applyMap model.LightToTemperatureMap
        |> applyMap model.TemperatureToHumidityMap
        |> applyMap model.HumidityToLocationMap


In [6]:
match run seedList "seeds: 79 14 55 13\n" with
| ParserResult.Failure (msg,_,_) -> failwith msg
| ParserResult.Success (section,_,_) ->
    section |> shouldEqual (SeedList [79L<seed>; 14L<seed>; 55L<seed>; 13L<seed>])

match run Model.parse "seed-to-soil map:\n50 98 2\n52 50 48\n" with
| ParserResult.Failure (msg,_,_) -> failwith msg
| ParserResult.Success (model,_,_) ->
    [79L<seed>; 14L<seed>; 55L<seed>; 13L<seed>]
    |> List.map (applyMap model.SeedToSoilMap)
    |> shouldEqual [81L<soil>; 14L<soil>; 57L<soil>; 13L<soil>]

let testInput = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""

match run Model.parse testInput with
| ParserResult.Failure (msg,_,_) -> failwith msg
| ParserResult.Success (model,_,_) ->
    model.Seeds |> shouldEqual [79L<seed>; 14L<seed>; 55L<seed>; 13L<seed>]
    
    model.Seeds
    |> List.map (Model.getLocation model)
    |> shouldEqual [82L<location>; 43L<location>; 86L<location>; 35L<location>]


In [7]:
open System.IO
let input = File.ReadAllText "input_05.txt"

let result = 
    match run Model.parse input with
    | ParserResult.Failure (msg,_,_) -> failwith msg
    | ParserResult.Success (model,_,_) ->
        model.Seeds
        |> List.map (Model.getLocation model)
        |> List.min

printfn "Result: %d" result 
result |> shouldEqual 31599214L<location> // In case I break it

Result: 31599214


## Part 2

In [11]:
let reinterpretSeedList (seeds: int64<seed> list) =
    seeds
    |> List.pairwise
    |> List.indexed
    |> Seq.choose (
        fun (i,(a,b)) -> 
            if i % 2 = 1 then
                None
            else
                Some (a, UMX.untag b)
    )

let applyMap2 tryGetCached setCached (ranges: MapRange<'src, 'dst>[]) (src: int64<'src>) : int64<'dst> =
    let index = 
        match tryGetCached src with
        | Some i -> i
        | None -> binarySearchBy mapOrder src ranges
    setCached index
    match index with
    | BeforeFirst -> UMX.cast<'src,'dst>src
    | AfterLast i
    | Match i
    | Between (i, _) -> 
        let range = ranges[i]
        let offset = UMX.untag<'src>(src - range.SourceStart)
        if offset < range.RangeSize then
            range.DestinationStart + UMX.tag<'dst>offset
        else
            UMX.cast<'src,'dst>src    

let calcResult2 model = 
    let mutable minValue = UMX.tag<location>Int64.MaxValue
    for (start, length) in (reinterpretSeedList model.Seeds) do
        let mutable remaining = length
        let mutable current = start
        while remaining > 0L do
            let location = Model.getLocation model current
            if location < minValue then
                minValue <- location
            remaining <- remaining - 1L
            current <- current + 1L<seed>

    minValue


In [12]:
let result2 = 
    match run Model.parse input with
    | ParserResult.Failure (msg,_,_) -> failwith msg
    | ParserResult.Success (model,_,_) ->
        calcResult2 model // 52 min!

In [13]:
printfn "Result2: %d" result2
result2 |> shouldEqual 20358599L<location> // In case I break it

Result2: 20358599
